# The Deutsch-Jozsa algorithm

**Inputs**: A black box $U_f$ which performs the transformation $|x\rangle|y\rangle\rightarrow|x\rangle|y\oplus f(x)\rangle$, for $x\in\left\{0,...,2^n-1\right\}$ and $f(x)\in\left\{0, 1\right\}$. It is promised that $f(x)$ is either *constant* for all values of $x$, or else $f(x)$ is *balanced*, that is, equal to $1$ for exactly half of all the possible $x$, and $0$ for the other half.

**Outputs**: $0$ if and only if $f$ is constant.

**Runtime**: One evaluation of $U_f$. Always succeeds.

**Procedure**:

1. $|0\rangle^{\otimes n}|1\rangle$ (initialize state)
2. $\rightarrow\frac{1}{\sqrt{2^n}}\sum_{x=0}^{2^n-1}|x\rangle\left[\frac{|0\rangle-|1\rangle}{\sqrt{2}}\right]$ (create superposition using Hadamard gates)
3. $\rightarrow\sum_x(-1)^{f(x)}|x\rangle\left[\frac{|0\rangle-|1\rangle}{\sqrt{2}}\right]$ (calculate function $f$ using $U_f$)
4. $\rightarrow\sum_z\sum_x\frac{(-1)^{x\cdot z+f(x)}|z\rangle}{\sqrt{2^n}}\left[\frac{|0\rangle-|1\rangle}{\sqrt{2}}\right]$, where $z\cdot x$ is the bitwise inner product of $x$ and $z$, modulo 2 (perform Hadamard transform)
5. $\rightarrow z$ (measure to obtain final output $z$)

## Example case

This $N$ qubit circuit determines whether the function

$$\begin{equation}
f(x)=\begin{cases}
1, & x~\mathrm{is~odd}\\
0, & x~\mathrm{is~even}
\end{cases},
\end{equation}$$

is a constant or balanced function for $0\leq x<2^N$.

In [16]:
import numpy as np
from qelvin import QCircuit, QRegister

N = 3

# Step 1
psi_input = QRegister(N, (1 << (N - 1)))
circ = QCircuit(psi_input)

# Step 2
for i in range(0, N):
    circ.h(i)

# Step 3
#circ.cx(N - 2, N - 1)
circ.cx(0, N-1)

# Step 4
for i in range(0, N - 1):
    circ.h(i)

print(circ)

circ.run()


# Step 5

psi_output = circ.state()

real0, imag0 = psi_output[0]
real1, imag1 = psi_output[1 << (N-1)]

is_constant = real0*real0 + imag0*imag0 + real1*real1 + imag1*imag1

if abs(is_constant) < 1e-9:
    print("Function is balanced")
elif abs(is_constant-1.0) < 1e-9:
    print("Function is constant")

      -------           -------  
     |       |         |       | 
q0---|   H   |----o----|   H   |----
     |       |    |    |       | 
      -------     |     -------  
      -------     |     -------  
     |       |    |    |       | 
q1---|   H   |----|----|   H   |----
     |       |    |    |       | 
      -------     |     -------  
      -------  -------           
     |       ||       |          
q2---|   H   ||   X   |-------------
     |       ||       |          
      -------  -------           

Function is balanced
